In [1]:
%load_ext autoreload
%autoreload 2
%pip install transformers trl

In [3]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

In [7]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("/content/cleaned_first_100_issues.csv")

# Print the column headings
print(df.columns.tolist())

['title', 'labels', 'body', 'issue_url', 'repository_url', 'language']


In [10]:
import os
import openai
import re

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expression to extract locations from input
    pattern = re.compile(r"FROM\s+(.+?)\s+TO\s+(.+)", re.IGNORECASE)
    match = pattern.search(user_input)
    if match:
        start_location = match.group(1).strip()
        end_location = match.group(2).strip()
        return start_location, end_location
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

# Run the chatbot in a loop
print("Route Description Chatbot. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use 'HEY TAKE ME FROM [start location] TO [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description:", route_description)


Route Description Chatbot. Type 'exit' to quit.


KeyboardInterrupt: Interrupted by user

In [15]:
import os
import openai
import re
import speech_recognition as sr

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expression to extract locations from input
    pattern = re.compile(r"FROM\s+(.+?)\s+TO\s+(.+)", re.IGNORECASE)
    match = pattern.search(user_input)
    if match:
        start_location = match.group(1).strip()
        end_location = match.group(2).strip()
        return start_location, end_location
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def record_audio():
    # Record audio using the microphone
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak now...")
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio, language='en-US')  # Adjust language as needed
        return text
    except sr.UnknownValueError:
        print("Sorry, I could not understand audio.")
        return ""
    except sr.RequestError:
        print("Sorry, there was a problem with the speech recognition service.")
        return ""

# Run the chatbot in a loop
print("Route Description Chatbot. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request (text/audio): ")

    if user_input.lower() == 'exit':
        print('Exiting')
        break

    if user_input.startswith('audio'):
        # Record audio
        user_input = record_audio()
        if not user_input:
            continue

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use 'HEY TAKE ME FROM [start location] TO [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description:", route_description)


Route Description Chatbot. Type 'exit' to quit.
Enter your request (text/audio): text
Invalid input format. Please use 'HEY TAKE ME FROM [start location] TO [end location]'.


KeyboardInterrupt: Interrupted by user

In [5]:
import os
import re
import wave
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
from googletrans import Translator  # For multilingual input
import openai

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expression to extract locations from input
    pattern = re.compile(r"FROM\s+(.+?)\s+TO\s+(.+)", re.IGNORECASE)
    match = pattern.search(user_input)
    if match:
        start_location = match.group(1).strip()
        end_location = match.group(2).strip()
        return start_location, end_location
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="text-davinci-003",  # Consider using a more descriptive model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def record_audio_sounddevice():
    duration = 5  # Adjust recording duration as needed
    fs = 44100  # Sample rate
    print("Speak now (recording for", duration, "seconds)...")

    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()

    print("Recording stopped.")

    # Save the recorded audio to a WAV file
    wav.write('recorded_audio.wav', fs, recording)

def transcribe_audio_whisper(audio_file_path):
    # Placeholder for Whisper API integration
    # Replace with your Whisper API integration logic
    # For example, using requests library to send audio data to Whisper API
    # and getting back the transcribed text

    # For the sake of example, assume audio_file_path contains audio data
    # and whisper_api_key is your Whisper API key
    whisper_api_key = ""

    # Example placeholder code (replace with actual Whisper API integration)
    with open(audio_file_path, 'rb') as f:
        audio_data = f.read()

    # Placeholder for Whisper API call
    # Example placeholder response:
    transcribed_text = "This is where your transcribed text would be."

    return transcribed_text

def translate_text(text, target_language="en"):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_language).text
    return translated_text

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        user_input = input("Enter your request (text/audio [language: en|es|fr...]): ")

        if user_input.lower() == 'exit':
            print('Exiting')
            break

        # Handle language specification (optional)
        language_code = None
        if ":" in user_input:
            parts = user_input.split(":")
            user_input = parts[0].strip()
            language_code = parts[1].strip().lower()

        if user_input.startswith('audio'):
            # Record audio using sounddevice
            record_audio_sounddevice()

            # Placeholder for Whisper API integration (currently using transcribe_audio_whisper placeholder)
            audio_file_path = "recorded_audio.wav"
            transcribed_text = transcribe_audio_whisper(audio_file_path)

            if not transcribed_text:
                print("Could not transcribe audio. Please try again.")
                continue

            user_input = transcribed_text

        # Optional: Translate text if language specified
        if language_code:
            user_input = translate_text(user_input, language_code)

        start_location, end_location = extract_locations(user_input)
        if not start_location or not end_location:
            print("Could not extract start and end locations from input. Please try again.")
            continue

        route_description = generate_route_description(start_location, end_location)
        print("Route Description:", route_description)

if __name__ == "__main__":
    main()


OSError: PortAudio library not found

In [9]:
from IPython.display import HTML
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import os
import re
import openai
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

def extract_locations(user_input):
    # Regular expression to extract locations from input
    pattern = re.compile(r"FROM\s+(.+?)\s+TO\s+(.+)", re.IGNORECASE)
    match = pattern.search(user_input)
    if match:
        start_location = match.group(1).strip()
        end_location = match.group(2).strip()
        return start_location, end_location
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def translate_text(text, target_language="en"):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_language).text
    return translated_text

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        user_input = input("Enter your request (text/audio [language: en|es|fr...]): ")

        if user_input.lower() == 'exit':
            print('Exiting')
            break

        # Handle language specification (optional)
        language_code = None
        if ":" in user_input:
            parts = user_input.split(":")
            user_input = parts[0].strip()
            language_code = parts[1].strip().lower()

        if user_input.startswith('audio'):
            # Display HTML/JS for recording audio in Colab
            display(HTML(AUDIO_HTML))

            # Evaluate JavaScript to get base64 encoded audio data
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            # Convert from webm to wav format using ffmpeg
            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            # Convert the byte stream to a format compatible with scipy.io.wavfile.read
            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            # Read wav data
            sr, audio = wav_read(io.BytesIO(riff))

            # Optional: Translate text if language specified
            if language_code:
                user_input = translate_text(user_input, language_code)

            # Extract locations from user input
            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            # Generate route description
            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            # Handle text input directly
            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            # Optional: Translate text if language specified
            if language_code:
                user_input = translate_text(user_input, language_code)

            # Generate route description
            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter your request (text/audio [language: en|es|fr...]): EXIT
Exiting


In [10]:
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import os
import re
import openai
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

def extract_locations(user_input):
    # Regular expression to extract locations from input
    pattern = re.compile(r"FROM\s+(.+?)\s+TO\s+(.+)", re.IGNORECASE)
    match = pattern.search(user_input)
    if match:
        start_location = match.group(1).strip()
        end_location = match.group(2).strip()
        return start_location, end_location
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def translate_text(text, target_language="en"):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_language).text
    return translated_text

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        user_input = input("Enter your request: ")

        # Handle language specification (optional)
        language_code = None
        if ":" in user_input:
            parts = user_input.split(":")
            user_input = parts[0].strip()
            language_code = parts[1].strip().lower()

        if input_type == 'audio':
            # Display HTML/JS for recording audio in Colab
            display(HTML(AUDIO_HTML))

            # Evaluate JavaScript to get base64 encoded audio data
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            # Convert from webm to wav format using ffmpeg
            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            # Convert the byte stream to a format compatible with scipy.io.wavfile.read
            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            # Read wav data
            sr, audio = wav_read(io.BytesIO(riff))

            # Optional: Translate text if language specified
            if language_code:
                user_input = translate_text(user_input, language_code)

            # Extract locations from user input
            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            # Generate route description
            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            # Handle text input directly
            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            # Optional: Translate text if language specified
            if language_code:
                user_input = translate_text(user_input, language_code)

            # Generate route description
            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: TEXT
Enter your request: मुझे कोझिकोड से चेन्नई ले चलो
Could not extract start and end locations from input. Please try again.


KeyboardInterrupt: Interrupted by user

In [12]:
import os
import openai
from googletrans import Translator
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

def extract_locations(user_input):
    # Initialize SpaCy model for English
    nlp_en = spacy.load("en_core_web_sm")

    # Detect language of user input
    translator = Translator()
    detected_lang = translator.detect(user_input).lang

    # Translate input to English if necessary
    if detected_lang != 'en':
        user_input = translator.translate(user_input, src=detected_lang, dest='en').text

    # Apply SpaCy NER to extract locations
    doc = nlp_en(user_input)
    locations = []
    for ent in doc.ents:
        if ent.label_ == "LOC":
            locations.append(ent.text)

    # Return start and end locations (first two locations found)
    if len(locations) >= 2:
        return locations[0], locations[1]
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def translate_text(text, target_language="en"):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_language).text
    return translated_text

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        user_input = input("Enter your request: ")

        # Handle language specification (optional)
        language_code = None
        if ":" in user_input:
            parts = user_input.split(":")
            user_input = parts[0].strip()
            language_code = parts[1].strip().lower()

        if language_code:
            # Translate user input to English if necessary
            user_input = translate_text(user_input, "en")

        if input_type == 'audio':
            # Display HTML/JS for recording audio in Colab
            display(HTML(AUDIO_HTML))

            # Evaluate JavaScript to get base64 encoded audio data
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            # Convert from webm to wav format using ffmpeg
            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            # Convert the byte stream to a format compatible with scipy.io.wavfile.read
            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            # Read wav data
            sr, audio = wav_read(io.BytesIO(riff))

        else:
            # Handle text input directly
            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            # Generate route description
            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: मुझे कोझिकोड से चेन्नई ले चलो


AttributeError: 'NoneType' object has no attribute 'group'

In [13]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def extract_locations(user_input):
    # Initialize SpaCy model for English
    nlp_en = spacy.load("en_core_web_sm")

    # Translate input to English using OpenAI
    translated_input = translate_text_openai(user_input, "en")

    # Apply SpaCy NER to extract locations
    doc = nlp_en(translated_input)
    locations = []
    for ent in doc.ents:
        if ent.label_ == "LOC":
            locations.append(ent.text)

    # Return start and end locations (first two locations found)
    if len(locations) >= 2:
        return locations[0], locations[1]
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        user_input = input("Enter your request: ")

        # Handle language specification (optional)
        language_code = None
        if ":" in user_input:
            parts = user_input.split(":")
            user_input = parts[0].strip()
            language_code = parts[1].strip().lower()

        if language_code:
            # Translate user input to English if necessary
            user_input = translate_text_openai(user_input, "en")

        if input_type == 'audio':
            # Display HTML/JS for recording audio in Colab
            display(HTML(AUDIO_HTML))

            # Evaluate JavaScript to get base64 encoded audio data
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            # Convert from webm to wav format using ffmpeg
            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            # Convert the byte stream to a format compatible with scipy.io.wavfile.read
            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            # Read wav data
            sr, audio = wav_read(io.BytesIO(riff))

        else:
            # Handle text input directly
            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            # Generate route description
            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: audio
Enter your request: मुझे कोझिकोड से चेन्नई ले चलो


AttributeError: 'int' object has no attribute 'split'

In [18]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var base64data = 0;
var reader;
var recorder, gumStream;

function startRecording() {
    var recordButton = document.createElement('button');
    recordButton.innerText = 'Recording... Click to stop';
    document.body.appendChild(recordButton);

    var handleSuccess = function(stream) {
        gumStream = stream;
        var options = {
            mimeType: 'audio/webm;codecs=opus'
        };
        recorder = new MediaRecorder(stream, options);
        recorder.ondataavailable = function(e) {
            var url = URL.createObjectURL(e.data);
            var preview = document.createElement('audio');
            preview.controls = true;
            preview.src = url;
            document.body.appendChild(preview);

            reader = new FileReader();
            reader.readAsDataURL(e.data);
            reader.onloadend = function() {
                base64data = reader.result;
            }
        };
        recorder.start();
    };

    recordButton.onclick = function() {
        if (recorder && recorder.state == "recording") {
            recorder.stop();
            gumStream.getAudioTracks()[0].stop();
            recordButton.innerText = "Saving the recording... Please wait!";
        }
    };

    navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
}

startRecording();
</script>
"""

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def extract_locations(user_input):
    # Initialize SpaCy models for multiple languages
    nlp_en = spacy.load("en_core_web_sm")
    nlp_fr = spacy.load("fr_core_news_sm")
    nlp_es = spacy.load("es_core_news_sm")
    nlp_hi = spacy.load("xx_ent_wiki_sm")  # Using the Hindi model for illustration

    # Translate input to English using OpenAI
    translated_input = translate_text_openai(user_input, "en")

    # Apply SpaCy NER to extract locations in English
    doc_en = nlp_en(translated_input)
    locations = []
    for ent in doc_en.ents:
        if ent.label_ == "LOC":
            locations.append(ent.text)

    # If no locations found in English, try French
    if not locations:
        doc_fr = nlp_fr(translated_input)
        for ent in doc_fr.ents:
            if ent.label_ == "LOC":
                locations.append(ent.text)

    # If still no locations found, try Spanish
    if not locations:
        doc_es = nlp_es(translated_input)
        for ent in doc_es.ents:
            if ent.label_ == "LOC":
                locations.append(ent.text)

    # If still no locations found, try Hindi
    if not locations:
        doc_hi = nlp_hi(translated_input)
        for ent in doc_hi.ents:
            if ent.label_ == "LOC":
                locations.append(ent.text)

    # Return start and end locations (first two locations found)
    if len(locations) >= 2:
        return locations[0], locations[1]
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            # Display HTML/JS for recording audio in Colab
            display(HTML(AUDIO_HTML))

            # Wait for audio recording to complete
            print("Speak your request. Recording will stop automatically.")
            eval_js("new Promise((resolve, reject) => setTimeout(() => resolve(base64data), 30000));")

            # Evaluate JavaScript to get base64 encoded audio data
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            # Convert from webm to wav format using ffmpeg
            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            # Convert the byte stream to a format compatible with scipy.io.wavfile.read
            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            # Read wav data
            sr, audio = wav_read(io.BytesIO(riff))

            # Convert audio to text
            user_input = " ".join([word.capitalize() for word in transcribe_audio(audio).split()])

        else:
            # Handle text input directly
            user_input = input("Enter your request: ")

        # Handle language specification (optional)
        language_code = None
        if ":" in user_input:
            parts = user_input.split(":")
            user_input = parts[0].strip()
            language_code = parts[1].strip().lower()

        if language_code:
            # Translate user input to English if necessary
            user_input = translate_text_openai(user_input, "en")

        # Extract locations from user input
        start_location, end_location = extract_locations(user_input)
        if not start_location or not end_location:
            print("Could not extract start and end locations from input. Please try again.")
            continue

        # Generate route description
        route_description = generate_route_description(start_location, end_location)
        print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: audio


Speak your request. Recording will stop automatically.


AttributeError: 'int' object has no attribute 'split'

In [23]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from langchain.pipeline import Pipeline
import faiss
import numpy as np

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy model for English
nlp_en = spacy.load("en_core_web_sm")

# Initialize langchain pipeline for translation
pipeline = Pipeline(source_lang="auto", target_lang="en")

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="gpt-3.5-turbo",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input):
    # Translate input to English using langchain
    translated_input = pipeline.translate(user_input)

    # Apply SpaCy NER to extract locations
    doc = nlp_en(translated_input)
    locations = []
    for ent in doc.ents:
        if ent.label_ == "LOC":
            locations.append(ent.text)

    # Return start and end locations (first two locations found)
    if len(locations) >= 2:
        return locations[0], locations[1]
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            # Display HTML/JS for recording audio in Colab
            display(HTML(AUDIO_HTML))

            # Evaluate JavaScript to get base64 encoded audio data
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            # Convert from webm to wav format using ffmpeg
            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            # Convert the byte stream to a format compatible with scipy.io.wavfile.read
            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            # Read wav data
            sr, audio = wav_read(io.BytesIO(riff))

            # Translate audio to text using OpenAI Whisper
            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

        else:
            user_input = input("Enter your request: ")

            # Handle language specification (optional)
            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                # Translate user input to English if necessary
                user_input = translate_text_openai(user_input, "en")

            # Extract locations from user input
            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            # Generate route description
            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'langchain.pipeline'

In [24]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy model for English
nlp_en = spacy.load("en_core_web_sm")

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input):
    doc = nlp_en(user_input)
    locations = []
    for ent in doc.ents:
        if ent.label_ == "LOC":
            locations.append(ent.text)
    if len(locations) >= 2:
        return locations[0], locations[1]
    else:
        return None, None

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: from kozhikode to kannur
Could not extract start and end locations from input. Please try again.
Enter 'text' for text input or 'audio' for audio input: Take me from kozhikode to kannur
Invalid input type. Please enter 'text' or 'audio'.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: Take me from kozhikode to kannur
Could not extract start and end locations from input. Please try again.
Enter 'text' for text input or 'audio' for audio input: exit
Exiting


In [29]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] =
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy model for English
nlp_en = spacy.load("en_core_web_sm")

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input):
    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = user_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Iterate over words to find locations
    for i in range(len(words)):
        if words[i] in direction_keywords:
            if i + 1 < len(words):
                if start_location is None:
                    start_location = words[i + 1].capitalize()  # Assuming location is in title case
                elif end_location is None:
                    end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            start_location, end_location = extract_locations(audio_data)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: take me from kannur to kozhikode
Invalid input type. Please enter 'text' or 'audio'.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: take me from kannur to kozhikode
Route Description: Sure! Here's a detailed driving route from Me (assuming starting point) to Kannur in Kerala:

1. Start from your location in Me.
2. Head southeast on Me Bypass Rd.
3. Turn left onto NH66/National Highway 66.
4. Continue on NH66, passing through towns like Thalassery and Mahe.
5. Once you reach Thalassery, stay on NH66 and continue towards Kannur.
6. Pass by landmarks such as Dharmadam and Kannadikkal.
7. Keep following NH66 until you reach your destination in Kannur.

Distance: The approximate driving distance between Me and Kannur is around 75-80 kilometers, depending on your exact starting point.

Please note that road conditions and traffic may vary, so I recommend c

KeyboardInterrupt: Interrupted by user

In [30]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy models
nlp_models = {
    "en": spacy.load("en_core_web_sm"),
    "fr": spacy.blank("fr"),  # French
    "es": spacy.blank("es"),  # Spanish
    "hi": spacy.blank("xx")   # Hindi (Note: xx doesn't have an actual model, it's a placeholder)
}

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language="en"):
    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = user_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Check if an appropriate SpaCy model is available for the input language
    if input_language in nlp_models:
        nlp = nlp_models[input_language]
        doc = nlp(user_input)

        for ent in doc.ents:
            if ent.label_ == "LOC":
                if start_location is None:
                    start_location = ent.text.capitalize()  # Assuming location is in title case
                elif end_location is None:
                    end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any(char >= 'ऀ' and char <= 'ॿ' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: TEXT
Enter your request: मुझे कोझिकोड से चेन्नई तक विस्तार से ले चलो
Could not extract start and end locations from input. Please try again.


KeyboardInterrupt: Interrupted by user

In [31]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] ="
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy models
nlp_en = spacy.load("en_core_web_sm")

# Initialize Google Translator
translator = Translator()

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language="en"):
    if input_language == "hi":
        # Translate Hindi input to English
        translated_input = translator.translate(user_input, dest="en").text
    else:
        translated_input = user_input

    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = translated_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Use English SpaCy model for extraction
    doc = nlp_en(translated_input)

    for ent in doc.ents:
        if ent.label_ == "LOC":
            if start_location is None:
                start_location = ent.text.capitalize()  # Assuming location is in title case
            elif end_location is None:
                end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any('\u0900' <= char <= '\u097f' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: TEXT
Enter your request: मुझे कोझिकोड से चेन्नई तक विस्तार से ले चलो
Could not extract start and end locations from input. Please try again.
Enter 'text' for text input or 'audio' for audio input: TAKE ME FROM KOZHIKODE TO CHENNAI in detail
Invalid input type. Please enter 'text' or 'audio'.
Enter 'text' for text input or 'audio' for audio input: TEXT
Enter your request: TAKE ME FROM KOZHIKODE TO CHENNAI in detail
Route Description: Sure, I can provide you with a detailed driving route from Me to Kozhikode in Kerala, India. Please note that the route provided is based on a general path and may vary based on current road conditions and traffic.

Starting Point: Me (Assuming current location)
Destination: Kozhikode

Route:
1. Start by heading south on [Your current location road] towards [Main Road].
2. Merge onto [Main Road] and continue straight for about 5 kilometers.
3. Take a slig

KeyboardInterrupt: Interrupted by user

In [32]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy models
nlp_en = spacy.load("en_core_web_sm")

# Initialize Google Translator
translator = Translator()

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language="en"):
    if input_language == "hi":
        # Translate Hindi input to English
        translated_input = translator.translate(user_input, dest="en").text
    else:
        translated_input = user_input

    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = translated_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Use English SpaCy model for extraction
    doc = nlp_en(translated_input)

    for ent in doc.ents:
        if ent.label_ == "LOC":
            if start_location is None:
                start_location = ent.text.capitalize()  # Assuming location is in title case
            elif end_location is None:
                end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_current_location():
    return input("Please enter your current location: ").strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any('\u0900' <= char <= '\u097f' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: TEXT
Enter your request: मुझे कोझिकोड से चेन्नई तक विस्तार से ले चलो
Could not extract start and end locations from input. Please try again.
Enter 'text' for text input or 'audio' for audio input: TAKE ME FROM KOZHIKODE TO CHENNAI in detail
Invalid input type. Please enter 'text' or 'audio'.
Enter 'text' for text input or 'audio' for audio input: TEXT
Enter your request: TAKE ME FROM KOZHIKODE TO CHENNAI in detail
Please enter your current location: NIT CALICUT
Route Description: Driving route from NIT CALICUT to Kozhikode:

1. Start at NIT CALICUT, Head south toward NIT Road - 200m
2. Turn left onto NIT Road - 500m
3. Continue onto National Highway 766/NH766 - 3.5km
4. Continue straight to stay on NH766 - 6.5km
5. Turn left onto Vadakara Bypass Rd in Kakkattil - 280m
6. Continue onto Kuttiady - Thamarassery - 26.7km
7. Turn right onto SH34 - 13.7km
8. Continue onto Calicut - Waynad 

KeyboardInterrupt: Interrupted by user

In [36]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy models
nlp_en = spacy.load("en_core_web_sm")

# Initialize Google Translator
translator = Translator()

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="gpt-3.5-turbo",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language="en"):
    if input_language == "fr":
        # Translate Hindi input to English
        translated_input = translator.translate(user_input, dest="en").text
    else:
        translated_input = user_input

    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = translated_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Use English SpaCy model for extraction
    doc = nlp_en(translated_input)

    for ent in doc.ents:
        if ent.label_ == "LOC":
            if start_location is None:
                start_location = ent.text.capitalize()  # Assuming location is in title case
            elif end_location is None:
                end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_current_location():
    return input("Please enter your current location: ").strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any('\u0900' <= char <= '\u097f' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request:  TAKE ME FROM KOZHIKODE TO CHENNAI in detail
Please enter your current location: Paragon 
Route Description: I'm sorry, but I cannot provide real-time driving directions as the route may change due to traffic conditions or road closures. 

However, I can suggest a general route from Paragon to Kozhikode for you to use as a reference:

1. Start at Paragon and head north on the main road.
2. Continue straight for about 3 kilometers until you reach a roundabout.
3. Take the second exit at the roundabout to continue on the main road.
4. Stay on the main road for approximately 10 kilometers, passing through various neighborhoods and smaller towns.
5. After this, you will reach a larger town where you will need to make a right turn onto a major highway leading towards Kozhikode.
6. Follow the highway signs towards Kozhikode, which is approximately 50 kilometers awa

In [6]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] ="
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy models
nlp_en = spacy.load("en_core_web_sm")

# Initialize Google Translator
translator = Translator()

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language="en"):
    if input_language == "fr":
        # Translate Hindi input to English
        translated_input = translator.translate(user_input, dest="en").text
    else:
        translated_input = user_input

    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = translated_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Use English SpaCy model for extraction
    doc = nlp_en(translated_input)

    for ent in doc.ents:
        if ent.label_ == "LOC":
            if start_location is None:
                start_location = ent.text.capitalize()  # Assuming location is in title case
            elif end_location is None:
                end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_current_location():
    current_location = input("Please enter your current location: ").strip()
    language_choice = input("Enter 'en' for English or 'fr' for French for the route description: ").strip().lower()
    return current_location, language_choice

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any('\u0900' <= char <= '\u097f' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location, language_choice = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)

            # Translate route description if needed
            if language_choice == 'fr':
                route_description = translator.translate(route_description, src='en', dest='fr').text

            print("Route Description:")
            print(route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location, language_choice = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)

            # Translate route description if needed
            if language_choice == 'fr':
                route_description = translator.translate(route_description, src='en', dest='fr').text

            print("Route Description:")
            print(route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: मुझे कोझिकोड से चेन्नई तक विस्तार से ले चलो
Could not extract start and end locations from input. Please try again.
Enter 'text' for text input or 'audio' for audio input:  TAKE ME FROM KOZHIKODE TO CHENNAI in detail
Invalid input type. Please enter 'text' or 'audio'.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request:  TAKE ME FROM KOZHIKODE TO CHENNAI in detail
Please enter your current location: NIT CALCIUT
Enter 'en' for English or 'fr' for French for the route description: fr
Route Description:
Bien sûr, voici une voie de conduite détaillée de NIT Calicut à Kozhikode:

1. Commencez au campus NIT Calicut et dirigez-vous vers l'est sur Nit Road.
2. Tournez à gauche sur NH 66 (Kannur Kozhikode-Mangalore Highway).
3. Continuez sur NH 66 sur environ 25 km.
4. Passez Thikkodi et continuez le NH 66.
5. Après environ 15 km, vous atte

In [7]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy models
nlp_en = spacy.load("en_core_web_sm")

# Initialize Google Translator
translator = Translator()

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language="en"):
    if input_language == "hi":
        # Translate Hindi input to English
        translated_input = translator.translate(user_input, dest="en").text
    else:
        translated_input = user_input

    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = translated_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Use English SpaCy model for extraction
    doc = nlp_en(translated_input)

    for ent in doc.ents:
        if ent.label_ == "LOC":
            if start_location is None:
                start_location = ent.text.capitalize()  # Assuming location is in title case
            elif end_location is None:
                end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_current_location():
    current_location = input("Please enter your current location: ").strip()
    language_choice = input("Enter 'en' for English or 'fr' for French for the route description: ").strip().lower()
    return current_location, language_choice

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any('\u0900' <= char <= '\u097f' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location, language_choice = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)

            # Translate route description if needed
            if language_choice == 'fr':
                route_description = translator.translate(route_description, src='en', dest='fr').text

            print("Route Description:")
            print(route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location, language_choice = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)

            # Translate route description if needed
            if language_choice == 'fr':
                route_description = translator.translate(route_description, src='en', dest='fr').text

            print("Route Description:")
            print(route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request:  TAKE ME FROM KOZHIKODE TO CHENNAI in detail
Please enter your current location: NIT CALICUT
Enter 'en' for English or 'fr' for French for the route description: fr
Route Description:
Bien sûr!Voici une route de conduite détaillée de NIT Calicut à Kozhikode:

1. Commencez à NIT Calicut, National Institute of Technology Calicut, Kozhikode, Kerala 673601.

2. Dirigez-vous vers le sud sur Nit Road vers la 3e Avenue.

3. Tournez à gauche sur la 3e Avenue.

4. Continuez directement pour rester sur la 3e Avenue.

5. Tournez à gauche sur Nellikode - Kuthiravattom Road.

6. Continuez directement sur Nellikode Road.

7. Tournez à droite sur Wayanad Road.

8. Continuez directement sur Kozhikode Bypass Road.

9. Suivez Kozhikode Bypass Road sur environ 14 km.

10. Prenez la sortie vers Perambra.

11. Mergez-vous sur Perambra - Kozhikode Road.

12. Continuez sur Perambra

In [8]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy models
nlp_en = spacy.load("en_core_web_sm")

# Initialize Google Translator
translator = Translator()

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language="en"):
    if input_language == "hi":
        # Translate Hindi input to English
        translated_input = translator.translate(user_input, dest="en").text
    else:
        translated_input = user_input

    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = translated_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Use English SpaCy model for extraction
    doc = nlp_en(translated_input)

    for ent in doc.ents:
        if ent.label_ == "LOC":
            if start_location is None:
                start_location = ent.text.capitalize()  # Assuming location is in title case
            elif end_location is None:
                end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_current_location():
    current_location = input("Please enter your current location: ").strip()
    language_choice = input("Enter 'en' for English or 'fr' for French for the route description: ").strip().lower()
    return current_location, language_choice

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any('\u0900' <= char <= '\u097f' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location, language_choice = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)

            # Translate route description if needed
            if language_choice == 'hi':
                route_description = translator.translate(route_description, src='en', dest='fr').text

            print("Route Description:")
            print(route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location, language_choice = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)

            # Translate route description if needed
            if language_choice == 'fr':
                route_description = translator.translate(route_description, src='en', dest='fr').text

            print("Route Description:")
            print(route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.


KeyboardInterrupt: Interrupted by user

In [11]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy models
nlp_en = spacy.load("en_core_web_sm")

# Initialize Google Translator
translator = Translator()

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language="en"):
    if input_language == "hi":
        # Translate Hindi input to English
        translated_input = translator.translate(user_input, dest="en").text
    else:
        translated_input = user_input

    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = translated_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Use English SpaCy model for extraction
    doc = nlp_en(translated_input)

    for ent in doc.ents:
        if ent.label_ == "LOC":
            if start_location is None:
                start_location = ent.text.capitalize()  # Assuming location is in title case
            elif end_location is None:
                end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_current_location():
    current_location = input("Please enter your current location: ").strip()
    language_choice = input("Enter 'en' for English or 'hi' for Hindi for the route description: ").strip().lower()
    return current_location, language_choice

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any('\u0900' <= char <= '\u097f' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location, language_choice = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)

            # Translate route description if needed
            if language_choice == 'hi':
                route_description = translator.translate(route_description, src='en', dest='hi').text

            print("Route Description:")
            print(route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location, language_choice = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location)

            # Translate route description if needed
            if language_choice == 'hi':
                route_description = translator.translate(route_description, src='en', dest='hi').text

            print("Route Description:")
            print(route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request:  TAKE ME FROM KOZHIKODE TO CHENNAI in detail
Please enter your current location: Kozhikode city mall
Enter 'en' for English or 'hi' for Hindi for the route description: en
Route Description:
Sure! Here is a detailed driving route from Kozhikode City Mall to Kozhikode:

1. Start at Kozhikode City Mall and head south on Mavoor Road towards Puthiyapalam Junction.
2. Continue straight on Mavoor Road and pass by the Malabar Gold & Diamonds showroom on your left.
3. After approximately 1.5 km, you will reach the Puthiyapalam Junction. Take a left turn onto Wayanad Road.
4. Continue on Wayanad Road for about 2.7 km until you reach the Arayidathupalam Junction. Turn right onto Y.M.C.A Cross Road.
5. Drive straight on Y.M.C.A Cross Road for about 1.2 km and then take a right turn onto Mavoor Road.
6. Continue on Mavoor Road for approximately 2.5 km. You will pass by P

KeyboardInterrupt: Interrupted by user

In [12]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy models
nlp_en = spacy.load("en_core_web_sm")

# Initialize Google Translator
translator = Translator()

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language="en"):
    if input_language == "hi":
        # Translate Hindi input to English
        translated_input = translator.translate(user_input, dest="en").text
    else:
        translated_input = user_input

    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = translated_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Use English SpaCy model for extraction
    doc = nlp_en(translated_input)

    for ent in doc.ents:
        if ent.label_ == "LOC":
            if start_location is None:
                start_location = ent.text.capitalize()  # Assuming location is in title case
            elif end_location is None:
                end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location, language_choice):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    route_description = response.choices[0].message.content.strip()

    # Translate route description if needed
    if language_choice == 'hi':
        route_description = translator.translate(route_description, src='en', dest='hi').text

    return route_description

def get_current_location():
    current_location = input("Please enter your current location: ").strip()
    language_choice = input("Enter 'en' for English or 'hi' for Hindi for the route description: ").strip().lower()
    return current_location, language_choice

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any('\u0900' <= char <= '\u097f' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location, language_choice = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location, language_choice)

            print("Route Description:")
            print(route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            current_location, language_choice = get_current_location()

            # Assuming the current location is needed for accurate route description
            start_location = current_location

            route_description = generate_route_description(start_location, end_location, language_choice)

            print("Route Description:")
            print(route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request:  TAKE ME FROM KOZHIKODE TO CHENNAI in detail
Please enter your current location: Kozhikode
Enter 'en' for English or 'hi' for Hindi for the route description: hi
Route Description:
निश्चित रूप से!चूंकि आपने कोझीकोड से कोझीकोड तक एक विस्तृत ड्राइविंग मार्ग का अनुरोध किया था, इसलिए मैं एक परिपत्र मार्ग प्रदान करूँगा जो शुरुआती बिंदु पर वापस समाप्त होता है।कृपया ध्यान दें कि यह मार्ग सबसे अधिक प्रत्यक्ष नहीं हो सकता है, लेकिन यह आपको कोझीकोड के आसपास एक सुंदर दौरे पर ले जाएगा।

शुरुआती बिंदु: कोझिकोड

1. कोझीकोड पर शुरू करें और मावूर रोड पर दक्षिण की ओर।
2. न्यू बस स्टैंड रोड पर बाएं मुड़ें।
3. स्टेडियम रोड पर सीधे जारी रखें।
4. इंदिरा गांधी रोड पर दाएं मुड़ें और सीधे जारी रखें।
5. एसएम स्ट्रीट पर बाएं मुड़ें।
6. एसएम स्ट्रीट पर जारी रखें और मंचिरा रोड पर दाएं मुड़ें।
7. मैनचिरा स्क्वायर के आसपास ड्राइव करें।
8. मोंचिरा रोड पर जारी रखें और फिर वाईएमसीए रोड पर दा

In [38]:
!pip install --upgrade googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=f4043b67f147e04ace6ee1016788f6974ed07270c8358e96db43bfd2f7e427ff
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [26]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy model for English
nlp_en = spacy.load("en_core_web_sm")

# Initialize Google Translator
translator = Translator()

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language):
    # Translate input to English using Google Translate
    translated_input = translator.translate(user_input, dest='en').text

    # Initialize SpaCy model for English
    nlp = nlp_en if input_language == "en" else spacy.blank("xx")  # Using English or blank for other languages

    # Apply SpaCy NER to extract locations
    doc = nlp(translated_input)
    locations = []
    for ent in doc.ents:
        if ent.label_ == "LOC":
            locations.append(ent.text)

    # Return start and end locations (first two locations found)
    if len(locations) >= 2:
        return locations[0], locations[1]
    else:
        return None, None

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any(char >= 'ऀ' and char <= 'ॿ' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                user_input = translate_text_openai(user_input, "en")

            start_location, end_location = extract_locations(user_input, "en")
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: मुझे कोझिकोड से चेन्नई तक विस्तार से ले चलो


AttributeError: 'NoneType' object has no attribute 'group'

In [27]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy model for English
nlp_en = spacy.load("en_core_web_sm")

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language):
    # Translate input to English using OpenAI
    translated_input = translate_text_openai(user_input, "en")

    # Initialize SpaCy model for English
    nlp = nlp_en if input_language == "en" else spacy.blank("xx")  # Using English or blank for other languages

    # Apply SpaCy NER to extract locations
    doc = nlp(translated_input)
    locations = []
    for ent in doc.ents:
        if ent.label_ == "LOC":
            locations.append(ent.text)

    # Return start and end locations (first two locations found)
    if len(locations) >= 2:
        return locations[0], locations[1]
    else:
        return None, None

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any(char >= 'ऀ' and char <= 'ॿ' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            user_input = input("Enter your request: ")

            start_location, end_location = extract_locations(user_input, "en")
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: TAKE ME FROM KOZHIKODE TO CHENNAI in detail


AttributeError: module 'openai' has no attribute 'Translate'

In [28]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] "
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

# Initialize SpaCy model for English
nlp_en = spacy.load("en_core_web_sm")

def translate_text_google(text, target_language="en"):
    translator = Translator()
    translation = translator.translate(text, dest=target_language)
    return translation.text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input, input_language):
    # Translate input to English using Google Translate
    translated_input = translate_text_google(user_input, "en")

    # Initialize SpaCy model for English
    nlp = nlp_en if input_language == "en" else spacy.blank("xx")  # Using English or blank for other languages

    # Apply SpaCy NER to extract locations
    doc = nlp(translated_input)
    locations = []
    for ent in doc.ents:
        if ent.label_ == "LOC":
            locations.append(ent.text)

    # Return start and end locations (first two locations found)
    if len(locations) >= 2:
        return locations[0], locations[1]
    else:
        return None, None

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any(char >= 'ऀ' and char <= 'ॿ' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            user_input = input("Enter your request: ")

            start_location, end_location = extract_locations(user_input, "en")
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: take me from kannur to kozhikode


AttributeError: 'NoneType' object has no attribute 'group'

In [21]:
!pip install faiss-cpu
!pip install langchain.pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement langchain.pipeline (from versions: none)
ERROR: No matching distribution found for langchain.pipeline


In [19]:
!pip install langchain faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [17]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var recordButton = document.createElement('button');
recordButton.innerText = 'Press to start recording';
document.body.appendChild(recordButton);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
        mimeType: 'audio/webm;codecs=opus'
    };
    recorder = new MediaRecorder(stream, options);
    recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

recordButton.onclick = function() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... Please wait!";
    }
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

</script>
"""

def translate_text_openai(text, target_language="en"):
    response = openai.Translate(
        model="text-davinci-003",
        inputs={
            "text": text,
            "to_language": target_language
        }
    )
    translated_text = response['translations'][0]['translated_text']
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": audio_data}
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def extract_locations(user_input):
    # Translate input to English using OpenAI
    translated_input = translate_text_openai(user_input, "en")

    # Placeholder for location extraction logic (to be implemented)
    # Here, you would implement logic to extract locations from translated_input

    return None, None  # Replace with actual location extraction logic

def generate_route_description(start_location, end_location):
    # Placeholder for route description generation (to be implemented)
    # Here, you would implement logic to generate route description

    return "Route description placeholder"

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            # Display HTML/JS for recording audio in Colab
            display(HTML(AUDIO_HTML))

            # Evaluate JavaScript to get base64 encoded audio data
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            # Convert from webm to wav format using ffmpeg
            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            # Convert the byte stream to a format compatible with scipy.io.wavfile.read
            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            # Read wav data
            sr, audio = wav_read(io.BytesIO(riff))

            # Translate audio to text using OpenAI Whisper
            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

        else:
            user_input = input("Enter your request: ")

            # Handle language specification (optional)
            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            if language_code:
                # Translate user input to English if necessary
                user_input = translate_text_openai(user_input, "en")

            # Extract locations from user input
            start_location, end_location = extract_locations(user_input)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            # Generate route description
            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: मुझे कोझिकोड से चेन्नई ले चलो


AttributeError: module 'openai' has no attribute 'Translate'

In [5]:
!pip install spacy

In [1]:
!pip install  ffmpeg-python

In [1]:
!pip uninstall sounddevice
!pip install sounddevice


Found existing installation: sounddevice 0.4.7
Uninstalling sounddevice-0.4.7:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/_sounddevice.py
    /usr/local/lib/python3.10/dist-packages/sounddevice-0.4.7.dist-info/*
    /usr/local/lib/python3.10/dist-packages/sounddevice.py
Proceed (Y/n)? y
  Successfully uninstalled sounddevice-0.4.7
  Using cached sounddevice-0.4.7-py3-none-any.whl (32 kB)


In [3]:
!pip install sounddevice

In [2]:
!pip install scipy

In [4]:
!pip install openai-whisper
!pip install pyaudio
!pip install googletrans

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: Could not build wheels for pyaudio, which is required to install pyproject.toml-based projects


In [16]:
import os
import openai
import re
import subprocess
import speech_recognition as sr

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expression to extract location names
    locations = re.findall(r"\b[A-Z][a-z]+\b", user_input)  # Matches capitalized words (potential location names)

    if len(locations) >= 2:
        start_location, end_location = locations[:2]
        return start_location, end_location
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def record_audio():
    # Record audio using the microphone
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Recording audio. Speak now...")
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio, language='en-US')
        return text
    except sr.UnknownValueError:
        print("Sorry, I could not understand audio.")
        return ""
    except sr.RequestError:
        print("Sorry, there was a problem with the speech recognition service.")
        return ""

# Run the chatbot in a loop
print("Route Description Chatbot. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request (text/audio): ")

    if user_input.lower() == 'exit':
        print('Exiting')
        break

    if user_input.lower().startswith('audio'):
        # Record audio
        user_input = record_audio()
        if not user_input:
            continue

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Could not extract start and end locations from input. Please try again.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description:", route_description)


Route Description Chatbot. Type 'exit' to quit.
Enter your request (text/audio): Text
Could not extract start and end locations from input. Please try again.
Enter your request (text/audio): kannur to kochi
Could not extract start and end locations from input. Please try again.
Enter your request (text/audio): from kannur to kozhikode
Could not extract start and end locations from input. Please try again.


KeyboardInterrupt: Interrupted by user

In [14]:
!pip install SpeechRecognition


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 25.9 MB/s eta 0:00:00


In [2]:
!pip install openai==0.28

In [2]:
!pip install pyarrow

In [ ]:
config = PPOConfig(
    model_name="lvwerra/gpt2-imdb",
    learning_rate=1.41e-5,
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

In [ ]:
def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds
